In [1]:
import sys
sys.path.append('../build/mechsystem')

from mass_spring import *

from pythreejs import *

In [2]:
def euclidean_distance(a, b):
    return ((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)**0.5

In [3]:
mss = MassSpringSystem3d()
mss.gravity = (0,0,-9.81)

scaling_factor = 3.0

# define positions
midMidpointPos = tuple(x * scaling_factor for x in (0.0, 0.0, 0.0))

leftFrontAnkerPos = tuple(x * scaling_factor for x in (-1.0, 0.5, 0.0))
rightFrontAnkerPos = tuple(x * scaling_factor for x in (1.0, 0.5, 0.0))
frontMidpointPos = tuple(x * scaling_factor for x in (0.0, 0.5, 0.0))
rightFrontUpPos = tuple(x * scaling_factor for x in (0.5, 0.5, 0.5))
leftFrontUpPos = tuple(x * scaling_factor for x in (-0.5, 0.5, 0.5))

leftBackAnkerPos = tuple(x * scaling_factor for x in (-1.0, -0.5, 0.0))
rightBackAnkerPos = tuple(x * scaling_factor for x in (1.0, -0.5, 0.0))
backMidpointPos = tuple(x * scaling_factor for x in (0.0, -0.5, 0.0))
rightBackUpPos = tuple(x * scaling_factor for x in (0.5, -0.5, 0.5))
leftBackUpPos = tuple(x * scaling_factor for x in (-0.5, -0.5, 0.5))

# define masses
mMidMidPoint = mss.add (Mass(10, midMidpointPos))

mLeftBackAnker = mss.add (Fix( leftBackAnkerPos ))
mRightBackAnker = mss.add (Fix( rightBackAnkerPos ))
mBackMidPoint = mss.add (Mass(10, backMidpointPos))
mRightBackUp = mss.add (Mass(10, rightBackUpPos))
mLeftBackUp = mss.add (Mass(10, leftBackUpPos))
fLeftBackAnker = mss.add (Fix( leftBackAnkerPos ))
fRightBackAnker = mss.add (Fix( rightBackAnkerPos ))

mLeftFrontAnker = mss.add (Fix( leftFrontAnkerPos ))
mRightFrontAnker = mss.add (Fix( rightFrontAnkerPos ))
mFrontMidPoint = mss.add (Mass(10, frontMidpointPos))
mRightFrontUp = mss.add (Mass(10, rightFrontUpPos))
mLeftFrontUp = mss.add (Mass(10, leftFrontUpPos))
fLeftFrontAnker = mss.add (Fix( leftFrontAnkerPos ))
fRightFrontAnker = mss.add (Fix( rightFrontAnkerPos ))



# connect front
mss.add (Spring(1, 100000, (mLeftFrontAnker, mFrontMidPoint)))
mss.add (Spring(1, 100000, (mRightFrontAnker, mFrontMidPoint)))
mss.add (Spring(euclidean_distance(frontMidpointPos, rightFrontUpPos), 10000, (mFrontMidPoint, mRightFrontUp)))
mss.add (Spring(euclidean_distance(frontMidpointPos, leftFrontUpPos), 10000, (mFrontMidPoint, mLeftFrontUp)))
mss.add (Spring(euclidean_distance(leftFrontUpPos, rightFrontUpPos), 10000, (mLeftFrontUp, mRightFrontUp)))
mss.add (Spring(euclidean_distance(leftFrontAnkerPos, leftFrontUpPos), 10000, (mLeftFrontAnker, mLeftFrontUp)))
mss.add (Spring(euclidean_distance(rightFrontAnkerPos, rightFrontUpPos), 10000, (mRightFrontAnker, mRightFrontUp)))
# connect back
mss.add (Spring(1, 100000, (mLeftBackAnker, mBackMidPoint)))
mss.add (Spring(1, 100000, (mRightBackAnker, mBackMidPoint)))
mss.add (Spring(euclidean_distance(backMidpointPos, rightBackUpPos), 10000, (mBackMidPoint, mRightBackUp)))
mss.add (Spring(euclidean_distance(backMidpointPos, leftBackUpPos), 10000, (mBackMidPoint, mLeftBackUp)))
mss.add (Spring(euclidean_distance(leftBackUpPos, rightBackUpPos), 10000, (mLeftBackUp, mRightBackUp)))
mss.add (Spring(euclidean_distance(leftBackAnkerPos, leftBackUpPos), 10000, (mLeftBackAnker, mLeftBackUp)))
mss.add (Spring(euclidean_distance(rightBackAnkerPos, rightBackUpPos), 10000, (mRightBackAnker, mRightBackUp)))
# connect top
mss.add (Spring(euclidean_distance(leftFrontUpPos, rightBackUpPos), 10000, (mLeftFrontUp, mRightBackUp)))
mss.add (Spring(euclidean_distance(leftFrontUpPos, leftBackUpPos), 10000, (mLeftFrontUp, mLeftBackUp)))
mss.add (Spring(euclidean_distance(rightFrontUpPos, rightBackUpPos), 10000, (mRightFrontUp, mRightBackUp)))

# connect bottom
mss.add (Spring(euclidean_distance(leftFrontAnkerPos, midMidpointPos), 10000, (mLeftFrontAnker, mMidMidPoint)))
mss.add (Spring(euclidean_distance(rightFrontAnkerPos, midMidpointPos), 10000, (mRightFrontAnker, mMidMidPoint)))
mss.add (Spring(euclidean_distance(leftBackAnkerPos, midMidpointPos), 10000, (mLeftBackAnker, mMidMidPoint)))
mss.add (Spring(euclidean_distance(rightBackAnkerPos, midMidpointPos), 10000, (mRightBackAnker, mMidMidPoint)))   
mss.add (Spring(euclidean_distance(backMidpointPos, midMidpointPos), 10000, (mBackMidPoint, mMidMidPoint)))
mss.add (Spring(euclidean_distance(frontMidpointPos, midMidpointPos), 10000, (mFrontMidPoint, mMidMidPoint)))

22

In [4]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [5]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [6]:
def force(i, t):
    # sinusoidal force in z direction on all masses

    amplitude = 100.0
    frequency = 10.0
    gravity = 9.81
    force_z = amplitude * math.sin(2 * math.pi * frequency * t) - gravity

    return (0, 0, force_z)
    # return (0, 0, -gravity)

In [7]:
from time import sleep
for i in range(10000):
    mss.gravity = force(i, i*0.02)
    mss.simulate (0.02, 100)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)

KeyboardInterrupt: 